In [ ]:
import pandas as pd
import ta

base_path = '../tmp-data/real-time-analysis/3rd/'

In [ ]:
df1 = pd.read_pickle(f'{base_path}btcusdt-range_bar.pkl')
dfcp1 = df1.copy()
print(len(dfcp1))
dfcp2 = df1.copy()
dfcp2.dropna(inplace=True)
print(len(dfcp2))
df_sample = dfcp2


In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter
from bokeh.layouts import column
from bokeh.models import Range1d, HoverTool
from bokeh.models.formatters import DatetimeTickFormatter

# add a HoverTool to the default tools
hover_tool = HoverTool(tooltips=[('x', '@x'), ('y', '@y'), ('value', '$y')])
hover_tool_main = HoverTool(tooltips=[("x", "@{%F}"), ("y", "@y{$0,0.00}")], formatters={"@{%F}": "datetime"})

# Create a ColumnDataSource object to hold the plot data
source = ColumnDataSource(data=dict(
    x=df_sample.index,
    y=df_sample['close'],
    bb_upper=df_sample['bb_upper'],
    bb_lower=df_sample['bb_lower'],
    average_adr=df_sample['average_adr'],
    macd=df_sample['macd'],
    macd_signal=df_sample['macd_signal'],
    macd_histogram=df_sample['macd_histogram'],
    rsi=df_sample['rsi'],
    vol=df_sample['volume']
))



# Create a Range1d object for the x-axis range shared by all three figures
x_range = Range1d(start=df_sample.index[0], end=df_sample.index[-1])

# Create a new figure object
p = figure(title='ss', x_axis_label='Date',
           y_axis_label='Price', width=1400, x_range=x_range)
# configure the x-axis to display datetime values
date_fmt = '%Y-%m-%d %H:%M:%S'
tick_formatter = DatetimeTickFormatter(days=date_fmt)
p.xaxis.formatter = tick_formatter
p.add_tools(hover_tool)

# Plot the Close values as a line
p.line(x='x', y='y', source=source, legend_label='Close')

# Plot the bb_upper and bb_lower values as shaded areas
p.varea(x='x', y1='bb_upper', y2='bb_lower', source=source, alpha=0.2, legend_label='Bollinger Bands')

p_macd = figure(title='MACD', x_axis_label='Date', y_axis_label='Value', width=1400, x_range=x_range)
p_macd.add_tools(hover_tool)
# Plot the average ADR as a line
p_macd.line(x='x', y='average_adr', source=source, color='orange', legend_label='Average ADR')

# Plot the MACD as two lines (macd and signal)
p_macd.line(x='x', y='macd', source=source, color='blue', legend_label='MACD')
p_macd.line(x='x', y='macd_signal', source=source, color='red', legend_label='Signal')

# Plot the MACD histogram as a bar chart
p_macd.vbar(x='x', top='macd_histogram', source=source, width=1.0, alpha=0.2, color='gray', legend_label='Histogram')

# Plot the RSI as a line in a separate figure
p_rsi = figure(title='RSI', x_axis_label='Date', y_axis_label='RSI Value', width=1400, x_range=x_range)
p_rsi.line(x='x', y='rsi', source=source, color='purple')
p_rsi.add_tools(hover_tool)

# Plot the Volume as a Vol in a separate figure
p_vol = figure(title='Volume', x_axis_label='Date', y_axis_label='Value', width=1400, x_range=x_range)
p_vol.line(x='x', y='vol', source=source, color='blue')
p_vol.add_tools(hover_tool)

show(column(p, p_vol, p_macd, p_rsi))

In [ ]:


# target = pd.to_datetime('2023-04-29 20:48:59.999000') 
# window_df = df1.loc[df1.index >= target]
print(len(window_df))
window_df

In [ ]:
# Calculate the difference between the current close and the last close
close_diff = window_df['close'] - window_df['close'].shift(1)
range_size = window_df['average_adr'] * 0.1
window_df['range_size'] = range_size
# Add the difference as a new column 'close_diff'
window_df['close_diff'] = close_diff
window_df.head(30)

In [ ]:
import numpy as np
# Set a tolerance level for comparing the two columns
tolerance = 1e-6
# Count how many times range_size and close_diff are within the tolerance level
count = np.isclose(window_df['range_size'], abs(window_df['close_diff']), rtol=tolerance).sum()
count_not_zero = (window_df['close_diff'] > 0).sum()
print(f'The range_size and close_diff columns are close in value {count} times. Out of {count_not_zero} were the diff is not zero')